### SQLChatMessageHistory
    - 채팅 메세지를 SQL 데이터베이스에 저장하는 클래스

**주요 특징**

    - SQL 데이터베이스를 사용해 채팅 기록을 영구적으로 저장
    - 세션 ID를 통한 대화 구분 및 관리 가능
    - SQLAlchemy를 사용해 다양한 데이터베이스 사용 가능
    - 메세지 CRUD 가능

    - session_id : 사용자 이름, 이메일 같이 고유 식별자
    - connection : 데이터베이스 연결을 지정하는 문자

In [12]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id="sql_history",
    connection="sqlite:///sqlite.db"
)

In [13]:
chat_message_history.add_user_message(
    "안녕? 만나서 반가워 나는 Bear야"
)

In [14]:
chat_message_history.add_ai_message(
    "안녕 Bear, 만나서 반가워"
)

In [15]:
chat_message_history.messages

[HumanMessage(content='안녕? 만나서 반가워 나는 Bear야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 Bear, 만나서 반가워', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕? 만나서 반가워 나는 Bear야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 Bear, 만나서 반가워', additional_kwargs={}, response_metadata={})]

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        
        MessagesPlaceholder(variable_name="chat_history"),
        
        ("human", "{question}")
    ]
)

chain = prompt | ChatOpenAI(temperature=0, model="gpt-4o") | StrOutputParser()

In [17]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db"
    )

In [18]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_field = [
    ConfigurableFieldSpec(
        id="user_id",                                   # 설정 값의 고유 식별자
        annotation=str,                                 # 설정 값의 데이터 타입
        name="USER ID",                                 # 설정값의 이름
        description="Unique identifier for a user",     # 설정값에 대한 설명
        default="",                                     # 기본값
        is_shared=True                                  # 여러 대화에서 공유되는 값인지 여부
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="CONVERSATION ID",
        description="Unique identifier for a conversation",
        default="",
        is_shared=True
    )
]

In [19]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,                       # 대화 기록을 가져오는 함수
    input_messages_key="question",          # 입력 메세지 키
    history_messages_key="chat_history",    # 대화 기록 메세지의 키
    history_factory_config=config_field     # 대화 기록 조회 시 참고할 파라미터
)

In [21]:
config = {"configurable" : {"user_id" : "user1", "conversation_id" : "conversation1"}}

chain_with_history.invoke({"question" : "안녕 반가워! 내 이름은 bear야"}, config)

'안녕하세요, Bear! 만나서 반가워요. 어떻게 도와드릴까요?'

In [23]:
config = {"configurable" : {"user_id" : "user1", "conversation_id" : "conversation2"}}

chain_with_history.invoke({"question" : "나는 성은이야"}, config)

'안녕하세요, 성은님! 만나서 반가워요. 오늘 어떻게 도와드릴까요?'

In [24]:
config = {"configurable" : {"user_id" : "user1", "conversation_id" : "conversation1"}}

chain_with_history.invoke({"question" : "내 이름이 뭐야?"}, config)

'당신의 이름은 Bear라고 하셨죠! 다른 질문이나 도움이 필요하시면 언제든지 말씀해 주세요.'